# Python Type Checking (Guide)
* https://realpython.com/python-type-checking/

In [1]:
# A little jupyter magic to allow continuing execution after exception (show exception). 
import traceback
from IPython.core.magic import register_cell_magic

@register_cell_magic('handle')
def handle(line, cell):
    try:
        exec(cell)
    except Exception:
        print(traceback.format_exc())

## Type Systems
### Dynamic Typing

In [2]:
if False:
    1 + "two"  # This line never runs, so no TypeError is raised
else:
    1 + 2

In [3]:
%%handle
1 + "two"  # Now this is type checked, and a TypeError is raised

Traceback (most recent call last):
  File "/tmp/ipykernel_261451/3202849731.py", line 8, in handle
    exec(cell)
  File "<string>", line 1, in <module>
TypeError: unsupported operand type(s) for +: 'int' and 'str'



In [4]:
thing = "Hello"
type(thing)

str

In [5]:
thing = 28.1
type(thing)

float

### Static Typing

The opposite of dynamic typing is static typing. Static type checks are performed without running the program. In most statically typed languages, for instance C and Java, this is done as your program is compiled.

Python will always [remain a dynamically typed language](https://www.python.org/dev/peps/pep-0484/#non-goals). However, [PEP 484](https://www.python.org/dev/peps/pep-0484/) introduced type hints, which make it possible to also do static type checking of Python code.

### Duck Typing

Duck typing is a concept related to dynamic typing, where the type or the class of an object is less important than the methods it defines. Using duck typing you do not check types at all. Instead you check for the presence of a given method or attribute.

In [6]:
class TheHobbit:
    def __len__(self):
        return 95022

the_hobbit = TheHobbit()
len(the_hobbit)

95022

## Hello Types

In [7]:
def headline(text, align=True):
    if align:
        return f"{text.title()}\n{'-' * len(text)}"
    else:
        return f" {text.title()} ".center(50, "o")

In [8]:
print(headline("python type checking"))

Python Type Checking
--------------------


In [9]:
print(headline("python type checking", align=False))

oooooooooooooo Python Type Checking oooooooooooooo


In [10]:
def headline(text: str, align: bool = True) -> str:
    if align:
        return f"{text.title()}\n{'-' * len(text)}"
    else:
        return f" {text.title()} ".center(50, "o")

The most common tool for doing type checking is [Mypy](http://mypy-lang.org/).

In [11]:
!mypy headlines.py

headlines.py:16: error: Argument "align" to "headline" has incompatible type "str"; expected "bool"
Found 1 error in 1 file (checked 1 source file)


In [12]:
!mypy headlines_fixed.py

Success: no issues found in 1 source file


## Pros and Cons

**Notable Cons**
* Type hints take developer time and effort to add. Even though it probably pays off in spending less time debugging, you will spend more time entering code.
* Type hints work best in modern Pythons. Annotations were introduced in Python 3.0, and it’s possible to use type comments in Python 2.7. Still, improvements like variable annotations and postponed evaluation of type hints mean that you’ll have a better experience doing type checks using Python 3.6 or even Python 3.7.
* Type hints introduce a slight penalty in start-up time. If you need to use the typing module the import time may be significant, especially in short scripts.

---
### Measuring Import Time

In [13]:
# Python 3.7+ (faster startup)
# Python 3.6  (slow startup)
!python -m timeit -n 1 -r 1 "import typing"

1 loop, best of 1: 1.22 msec per loop


<br/>

Expect something like:
```bash
python3.6 -m timeit -n 1 -r 1 "import typing"
```
```
1 loops, best of 1: 9.77 msec per loop
```

<br/>

```bash
python3.7 -m timeit -n 1 -r 1 "import typing"
```
```
1 loop, best of 1: 1.97 msec per loop
```

In Python 3.7 there is also a new command line option that can be used to figure out how much time imports take. Using `-X importtime` you’ll get a report about all imports that are made:

In [14]:
!python3 -X importtime import_typing.py

import time: self [us] | cumulative | imported package
import time:       238 |        238 |   _io
import time:        43 |         43 |   marshal
import time:       303 |        303 |   posix
import time:       559 |       1142 | _frozen_importlib_external
import time:        98 |         98 |   time
import time:       279 |        376 | zipimport
import time:        62 |         62 |     _codecs
import time:       566 |        628 |   codecs
import time:       429 |        429 |   encodings.aliases
import time:       645 |       1700 | encodings
import time:       231 |        231 | encodings.utf_8
import time:       182 |        182 | _signal
import time:       255 |        255 | encodings.latin_1
import time:        49 |         49 |     _abc
import time:       267 |        315 |   abc
import time:       270 |        585 | io
import time:        64 |         64 |       _stat
import time:       272 |        336 |     stat
import time:      1044 |       1044 |     _collections_abc
im

* If you’ll read the report closely you can notice that around half of this time is spent on importing the `collections.abc` and `re` modules which typing depends on.

---

A few rules of thumb on whether to add types to your project are:
* If you are just beginning to learn Python, you can safely wait with type hints until you have more experience.
* Type hints add little value in short throw-away scripts.
* ⚠️ In libraries that will be used by others, especially ones published on PyPI, type hints add a lot of value. Other code using your libraries need these type hints to be properly type checked itself. For examples of projects using type hints see `cursive_re`, `black`, our own Real Python Reader, and Mypy itself.
* ⚠️ In bigger projects, type hints help you understand how types flow through your code, and are highly recommended. Even more so in projects where you cooperate with others.

In his excellent article [The State of Type Hints in Python](https://bernat.tech/posts/the-state-of-type-hints-in-python/) Bernát Gábor recommends that “type hints should be used whenever unit tests are worth writing.” Indeed, type hints play a similar role as tests in your code: they help you as a developer write better code.

## Annotations

### Function Annotations
E.g.:
```python
def func(arg: arg_type, optarg: arg_type = default) -> return_type:
    ...
```

In [15]:
import math

def circumference(radius: float) -> float:
    return 2 * math.pi * radius

In [16]:
circumference(1.23)

7.728317927830891

In [17]:
circumference.__annotations__

{'radius': float, 'return': float}

**MyPy**

Sometimes you might be confused by how Mypy is interpreting your type hints. For those cases there are special Mypy expressions: `reveal_type()` and `reveal_locals()`. You can add these to your code before running Mypy, and Mypy will dutifully report which types it has inferred. As an example, save the following code to `reveal.py`:

In [18]:
!mypy reveal.py

reveal.py:5: note: Revealed type is "builtins.float"
reveal.py:11: note: Revealed local types are:
reveal.py:11: note:     circumference: builtins.float
reveal.py:11: note:     radius: builtins.int


> **Note:** The reveal expressions are only meant as a tool helping you add types and debug your type hints. If you try to run the `reveal.py` file as a Python script it will crash with a `NameError` since `reveal_type()` is not a function known to the Python interpreter.

### Variable Annotations

Sometimes the type checker needs help in figuring out the types of variables as well. Variable annotations were defined in PEP 526 and introduced in Python 3.6.

In [19]:
pi: float = 3.142

def circumference(radius: float) -> float:
    return 2 * pi * radius

> **Note:** Static type checkers are more than able to figure out that `3.142` is a `float`, so in this example the annotation of `pi` is not necessary. As you learn more about the Python type system, you’ll see more relevant examples of variable annotations.

In [20]:
circumference(1)

__annotations__  # Annotations of local variables.

{'pi': float}

You’re allowed to annotate a variable without giving it a value. This adds the annotation to the `__annotations__` dictionary, while the variable remains undefined:

In [21]:
%%handle
nothing: str
nothing

Traceback (most recent call last):
  File "/tmp/ipykernel_261451/3202849731.py", line 8, in handle
    exec(cell)
  File "<string>", line 2, in <module>
NameError: name 'nothing' is not defined



In [22]:
__annotations__

{'pi': float}

### Type Comments

Annotations were introduced in Python 3, and they’ve not been backported to Python 2. This means that if you’re writing code that needs to support legacy Python, you can’t use annotations.

In [23]:
import math

def circumference(radius):
    # type: (float) -> float
    return 2 * math.pi * radius

Type comments are handled directly by the type checker, so these types are **not available** in the `__annotations__` dictionary:

In [24]:
circumference.__annotations__

{}

In [25]:
def headline(text, width=80, fill_char="-"):
    # type: (str, int, str) -> str
    return f" {text.title()} ".center(width, fill_char)

print(headline("type comments work", width=40))

---------- Type Comments Work ----------


<br/>

See `headlines_comments.py`:
```python
# headlines.py

def headline(
    text,           # type: str
    width=80,       # type: int
    fill_char="-",  # type: str
):                  # type: (...) -> str

    return f" {text.title()} ".center(width, fill_char)


print(headline("type comments work", width=40))
```

In [26]:
!python headlines_comments.py

---------- Type Comments Work ----------


In [27]:
!mypy headlines_comments.py

Success: no issues found in 1 source file


In [28]:
# Type comments for variables:
pi = 3.142  # type: float

### So, Type Annotations or Type Comments?
In short: **Use annotations if you can, use type comments if you must.**


There is also hidden option number three: **stub files**. You will learn about these later, when we discuss adding types to third party libraries.

## Playing With Python Types, Part 1

### Example: A Deck of Cards
See `game.py`.

In [29]:
!python game.py

P1: ♡Q ♢9 ♡3 ♠K ♠5 ♡J ♠3 ♠9 ♡A ♣J ♢2 ♡7 ♢Q
P2: ♢A ♡K ♠8 ♡4 ♠4 ♣3 ♡9 ♣9 ♠A ♣A ♢7 ♢10 ♣10
P3: ♣5 ♣Q ♠J ♢6 ♢3 ♣8 ♣2 ♣K ♣6 ♢5 ♠10 ♠6 ♡2
P4: ♡10 ♢4 ♢K ♡6 ♠7 ♢8 ♢J ♣4 ♠Q ♠2 ♡5 ♣7 ♡8


You will see how to extend this example into a more interesting game as we move along.

### Sequences and Mappings

In [30]:
name: str = "Guido"
pi: float = 3.142
centered: bool = False

In [31]:
names: list = ["Guido", "Jukka", "Ivan"]
version: tuple = (3, 7, 1)
options: dict = {"centered": False, "capitalize": True}

In [32]:
from typing import Dict, List, Tuple

names: List[str] = ["Guido", "Jukka", "Ivan"]
version: Tuple[int, int, int] = (3, 7, 1)
options: Dict[str, bool] = {"centered": False, "capitalize": True}

In [33]:
def create_deck(shuffle: bool = False) -> List[Tuple[str, str]]:
    """Create a new deck of 52 cards"""
    deck = [(s, r) for r in RANKS for s in SUITS]

    if shuffle:
        random.shuffle(deck)

    return deck

In many cases your functions will expect some kind of sequence, and not really care whether it is a `list` or a `tuple`. In these cases you should use `typing.Sequence` when annotating the function argument:

In [34]:
from typing import List, Sequence

def square(elems: Sequence[float]) -> List[float]:
    return [x**2 for x in elems]

⚠️ Using `Sequence` is an example of using **duck typing**. A `Sequence` is anything that supports `len()` and `.__getitem__()`, **independent of its actual type**.

### Type Aliases

The type hints might become quite oblique when working with nested types like the deck of cards. You may need to stare at `List[Tuple[str, str]]` a bit before figuring out that it matches our representation of a deck of cards.

Now consider how you would annotate `deal_hands()`:

In [35]:
def deal_hands(
    deck: List[Tuple[str, str]]
) -> Tuple[
    List[Tuple[str, str]],
    List[Tuple[str, str]],
    List[Tuple[str, str]],
    List[Tuple[str, str]],
]:
    """Deal the cards in the deck into four hands"""

    return (deck[0::4], deck[1::4], deck[2::4], deck[3::4])

That’s just terrible!

Recall that type annotations are regular Python expressions. That means that you can define your own type aliases by assigning them to new variables. You can for instance create `Card` and `Deck` type aliases:

In [36]:
from typing import List, Tuple

Card = Tuple[str, str]
Deck = List[Card]

In [37]:
def deal_hands(deck: Deck) -> Tuple[Deck, Deck, Deck, Deck]:
    """Deal the cards in the deck into four hands"""

    return (deck[0::4], deck[1::4], deck[2::4], deck[3::4])

In [38]:
from typing import List, Tuple
Card = Tuple[str, str]
Deck = List[Card]

# Inspect what `Deck` represents by printing it:
Deck

typing.List[typing.Tuple[str, str]]

### Functions Without Return Values

In [39]:
def play(player_name):
    print(f"{player_name} plays")

ret_val = play("Jacob")

Jacob plays


In [40]:
print(ret_val)

None


See `play.py`.

In [41]:
!mypy play.py

play.py:8: error: "play" does not return a value
Found 1 error in 1 file (checked 1 source file)


Note that being explicit about a function not returning anything is different from not adding a type hint about the return value:

In [42]:
!mypy play_not_annotated.py

Success: no issues found in 1 source file


As a more exotic case, note that you can also **annotate functions that are never expected to return normally (e.g. exceptions)**. 

This is done using `NoReturn`:

In [43]:
from typing import NoReturn

def black_hole() -> NoReturn:
    raise Exception("There is no going back ...")

### Example: Play Some Cards

See `game_v2.py`

In [44]:
!python game_v2.py

P4: ♢2   P1: ♡3   P2: ♠7   P3: ♣5   
P4: ♠A   P1: ♢6   P2: ♢8   P3: ♣A   
P4: ♠9   P1: ♡10  P2: ♣J   P3: ♢3   
P4: ♢9   P1: ♠4   P2: ♡2   P3: ♣9   
P4: ♣4   P1: ♠K   P2: ♡5   P3: ♠3   
P4: ♢10  P1: ♢4   P2: ♡A   P3: ♠10  
P4: ♣10  P1: ♢5   P2: ♢J   P3: ♣8   
P4: ♢7   P1: ♠5   P2: ♣7   P3: ♣K   
P4: ♡9   P1: ♠2   P2: ♣3   P3: ♢K   
P4: ♣6   P1: ♡J   P2: ♠8   P3: ♡K   
P4: ♡7   P1: ♢A   P2: ♡4   P3: ♠6   
P4: ♣Q   P1: ♠Q   P2: ♡6   P3: ♣2   
P4: ♡8   P1: ♡Q   P2: ♢Q   P3: ♠J   


### The Any Type

`choose()` works for both lists of names and lists of cards (and any other sequence for that matter). One way to add type hints for this would be the following:

In [45]:
import random
from typing import Any, Sequence

def choose(items: Sequence[Any]) -> Any:
    return random.choice(items)

While Mypy will correctly infer that names is a list of strings, *that information is lost* after the call to `choose()` *because* of the use of the `Any` type:

In [46]:
!mypy choose.py

choose.py:13: note: Revealed type is "builtins.list[builtins.str*]"
choose.py:18: note: Revealed type is "Any"


## Type Theory

See also: [PEP 483 "The Theory of Type Hints"](https://www.python.org/dev/peps/pep-0483/).

Read the section: https://realpython.com/python-type-checking/#type-theory.

### Subtypes

Formally, we say that a type T is a subtype of U if the following two conditions hold:
* Every value from T is also in the set of values of U type.
* Every function from U type is also in the set of functions of T type.

In [47]:
int(False)

0

In [48]:
int(True)

1

In [49]:
True + True

2

In [50]:
issubclass(bool, int)

True

In [51]:
def double(number: int) -> int:
    return number * 2

print(double(True))  # Passing in bool instead of int

2


> Subtypes are somewhat related to subclasses. In fact all *subclasses corresponds to subtypes*, and bool is a subtype of int because bool is a subclass of int. *However, there are also subtypes that do not correspond to subclasses*. For instance int is a subtype of float, but int is not a subclass of float.

### Covariant, Contravariant, and Invariant

* Wikipedia is good on this: https://en.wikipedia.org/wiki/Covariance_and_contravariance_%28computer_science%29

Some examples:

* `Tuple` is **covariant**. This means that it preserves the type hierarchy of its item types: `Tuple[bool]` is a subtype of `Tuple[int]` because `bool` is a subtype of `int`.
* `List` is **invariant**. Invariant types give no guarantee about subtypes. While all values of `List[bool]` are values of `List[int]`, you can append an int to `List[int]` and not to `List[bool]`. In other words, the second condition for subtypes does not hold, and `List[bool]` is not a subtype of `List[int]`.
* `Callable` is **contravariant** in its arguments. This means that it reverses the type hierarchy. You will see how `Callable` works later, but for now think of `Callable[[T], ...]` as a function with its only argument being of type `T`. An example of a `Callable[[int], ...]` is the `double()` function defined above. Being contravariant means that if a function operating on a `bool` is expected, then a function operating on an `int` would be acceptable.


### Gradual Typing and Consistent Types

Gradual typing is essentially made possible by the `Any` type.

Somehow `Any` sits both at the top and at the bottom of the type hierarchy of subtypes. Any type behaves as if it is a subtype of `Any`, and `Any` behaves as if it is a subtype of any other type.

Looking at the definition of subtypes above this is not really possible. Instead we talk about **consistent types**.

The type `T` is consistent with the type `U` if: 
* `T` is a subtype of `U`, or 
* either `T` or `U` is `Any`.

## Playing With Python Types, Part 2

Recall that you were trying to annotate the general `choose()` function. The problem with using Any is that you are needlessly losing type information. 

*You know that if you pass a list of strings to `choose()`, it will return a string*. Below you’ll see how to express this using **type variables**.


### Type Variables

A type variable is a special variable that can take on any type, depending on the situation.

```python
Choosable = TypeVar("Choosable")
```

> A type variable must be defined using `TypeVar` from the `typing` module. When used, a type variable *ranges over all possible types and takes the most specific type possible*.

In [52]:
!mypy choose.py

choose.py:13: note: Revealed type is "builtins.list[builtins.str*]"
choose.py:18: note: Revealed type is "Any"


In [53]:
!mypy choose_with_typevar.py

choose_with_typevar.py:14: note: Revealed type is "builtins.list[builtins.str*]"
choose_with_typevar.py:18: note: Revealed type is "builtins.str*"


Consider this example, especially the last two cases:
```python
# choose_examples.py

from choose_with_typevar import choose

reveal_type(choose(["Guido", "Jukka", "Ivan"]))
reveal_type(choose([1, 2, 3]))
reveal_type(choose([True, 42, 3.14]))
reveal_type(choose(["Python", 3, 7]))
```

In [54]:
!mypy choose_examples.py

choose_with_typevar.py:14: note: Revealed type is "builtins.list[builtins.str*]"
choose_with_typevar.py:18: note: Revealed type is "builtins.str*"
choose_examples.py:5: note: Revealed type is "builtins.str*"
choose_examples.py:6: note: Revealed type is "builtins.int*"
choose_examples.py:7: note: Revealed type is "builtins.float*"
choose_examples.py:8: note: Revealed type is "builtins.object*"


Note that none of these examples raised a type error. Is there a way to tell the type checker that `choose()` should accept both strings and numbers, but not both at the same time?

You can constrain type variables by listing the acceptable types:
```python
Choosable = TypeVar("Choosable", str, float)
```

In [55]:
!mypy choose_with_typevar_constrained.py

choose_with_typevar_constrained.py:13: note: Revealed type is "builtins.str*"
choose_with_typevar_constrained.py:14: note: Revealed type is "builtins.float*"
choose_with_typevar_constrained.py:15: note: Revealed type is "builtins.float*"
choose_with_typevar_constrained.py:16: error: Value of type variable "Choosable" of "choose" cannot be "object"
choose_with_typevar_constrained.py:16: note: Revealed type is "builtins.object*"
Found 1 error in 1 file (checked 1 source file)


In our card game we want to restrict `choose()` to be used for `str` and `Card`:
```python
Choosable = TypeVar("Choosable", str, Card)

def choose(items: Sequence[Choosable]) -> Choosable:
    ...
```

### Duck Types and Protocols

One way to categorize type systems is by whether they are nominal or structural:
* In a **nominal** system, comparisons between types are based on names and declarations. The Python type system is mostly nominal, where an int can be used in place of a float because of their subtype relationship.
* In a **structural** system, comparisons between types are based on structure. You could define a structural type `Sized` that includes all instances that define `.__len__()`, irrespective of their nominal type.

> There is ongoing work to bring a full-fledged structural type system to Python through [PEP 544](https://www.python.org/dev/peps/pep-0544/) which aims at adding a concept called **protocols**. Most of PEP 544 is already implemented in [Mypy](https://mypy.readthedocs.io/en/latest/protocols.html) though.

A protocol specifies one or more methods that must be implemented. For example, all classes defining `.__len__()` fulfill the `typing.Sized` protocol. We can therefore annotate `len()` as follows:

In [56]:
from typing import Sized

def len(obj: Sized) -> int:
    return obj.__len__()

Other examples of protocols defined in the typing module include: 
* `Container`, 
* `Iterable`, 
* `Awaitable`, 
* `ContextManager`.

You can also define your own protocols. 

This is done by inheriting from `Protocol` and defining the function signatures (with empty function bodies) that the protocol expects. The following example shows how `len()` and `Sized` could have been implemented:

In [57]:
from typing_extensions import Protocol

class Sized(Protocol):
    def __len__(self) -> int: ...

def len(obj: Sized) -> int:
    return obj.__len__()

⚠️ At the time of writing the support for self-defined protocols is still experimental and only available through the `typing_extensions` module. This module must be explicitly installed from PyPI by doing `pip install typing-extensions`.

* Further reading: https://github.com/python/typing/tree/master/typing_extensions.
* See also Python typing docs: https://typing.readthedocs.io/en/latest/.

😊 From Python 3.8, available from `typing` directly! 
* https://docs.python.org/3.8/library/typing.html#typing.Protocol

### The Optional Type

In [58]:
from typing import Sequence, Optional

def player_order(
    names: Sequence[str], start: Optional[str] = None
) -> Sequence[str]:
    ...

See also `player_order.py`:

In [59]:
!mypy player_order.py

player_order.py:10: error: Unsupported left operand type for + ("Sequence[str]")
Found 1 error in 1 file (checked 1 source file)


**Note:** Not getting the same error as in the tutorial's example:
```
player_order.py:8: error: Argument 1 to "index" of "list" has incompatible
                          type "Optional[str]"; expected "str"
```

**Note:** The use of None for optional arguments is so common that Mypy handles it automatically. Mypy assumes that a default argument of None indicates an optional argument even if the type hint does not explicitly say so. You could have used the following:

```python
def player_order(names: Sequence[str], start: str = None) -> Sequence[str]:
    ...
```
If you don’t want Mypy to make this assumption you can turn it off with the `--no-implicit-optional` command line option.

### Example: The Object(ive) of the Game

See: `game_oo.py`.

### Type Hints for Methods

In [60]:
class Card:
    SUITS = "♠ ♡ ♢ ♣".split()
    RANKS = "2 3 4 5 6 7 8 9 10 J Q K A".split()

    def __init__(self, suit: str, rank: str) -> None:
        self.suit = suit
        self.rank = rank

    def __repr__(self) -> str:
        return f"{self.suit}{self.rank}"

### Classes as Types

There is a correspondence between classes and types. 

For example, **all instances of the `Card` class together form the `Card` type**. 

In [61]:
class Deck:
    def __init__(self, cards: List[Card]) -> None:
        self.cards = cards

⚠️ **Notable special situation**

Mypy is able to connect your use of `Card` in the annotation with the definition of the `Card` class.

**This doesn’t work as cleanly though when you need to refer to the class currently being defined**. For example, the `Deck.create()` class method returns an object with type `Deck`. However, you can’t simply add `-> Deck` as the `Deck` class is not yet fully defined.

Instead, you are allowed to **use string literals** in annotations. These strings will only be evaluated by the type checker later, and can therefore contain self and forward references. The `.create()` method should use such string literals for its types:

In [62]:
class Deck:
    @classmethod
    def create(cls, shuffle: bool = False) -> "Deck":
        """Create a new deck of 52 cards"""
        cards = [Card(s, r) for r in Card.RANKS for s in Card.SUITS]
        if shuffle:
            random.shuffle(cards)
        return cls(cards)

Usually annotations are not used at runtime. This has given wings to the idea of postponing the evaluation of annotations. Instead of evaluating annotations as Python expressions and storing their value, the proposal is to store the string representation of the annotation and only evaluate it when needed.

Such functionality is planned to become standard in the still mythical Python 4.0. However, in Python 3.7 and later, forward references are available through a `__future__` import:

**Note:** This is standard from Python 3.10+.

In [63]:
from __future__ import annotations

class Deck:
    @classmethod
    def create(cls, shuffle: bool = False) -> Deck:
        ...

### Returning `self` or `cls`

There is one case where you might want to annotate `self` or `cls`, though. 

Consider what happens if you have a **superclass that other classes inherit from, and which has methods that return `self` or `cls`**:

In [64]:
# dogs.py

from datetime import date


class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls, name: str) -> "Animal":
        return cls(name, date.today())

    def twin(self, name: str) -> "Animal":
        cls = self.__class__
        return cls(name, self.birthday)


class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")


fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()

Fido says woof!
Pluto says woof!


While the code runs without problems, Mypy will flag a problem:

In [65]:
!mypy dogs.py

dogs.py:27: error: "Animal" has no attribute "bark"
dogs.py:28: error: "Animal" has no attribute "bark"
Found 2 errors in 1 file (checked 1 source file)


The issue is that even though the inherited `Dog.newborn()` and `Dog.twin()` methods will return a `Dog` the annotation says that they return an `Animal`.

In cases like this you want to be more careful to make sure the annotation is correct. The return type should match the type of `self` or the instance type of `cls`. This can be done **using type variables** that keep track of what is actually passed to `self` and `cls`:

In [66]:
# dogs_fixed.py

from datetime import date
from typing import Type, TypeVar

TAnimal = TypeVar("TAnimal", bound="Animal")  # Note `bound`.

class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls: Type[TAnimal], name: str) -> TAnimal:  # Note Type[...] - typing's equivalent of type(...).
        return cls(name, date.today())

    def twin(self: TAnimal, name: str) -> TAnimal:
        cls = self.__class__
        return cls(name, self.birthday)

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()

Fido says woof!
Pluto says woof!


In [67]:
!mypy dogs_fixed.py

Success: no issues found in 1 source file


#### ‼️ There are a few things to note in this example:

* The type variable `TAnimal` is used to denote that return values might be instances of subclasses of `Animal`.
* ⚠️ We specify that `Animal` is an upper bound for `TAnimal`. Specifying bound means that `TAnimal` will only be `Animal` or one of its subclasses. This is needed to properly restrict the types that are allowed.
* ⚠️ The `typing.Type[]` construct is the typing equivalent of `type()`. You need it to note that the class method expects a class and returns an instance of that class.

### Annotating `*args` and `**kwargs`

In the object oriented version of the game, we added the option to name the players on the command line. This is done by listing player names after the name of the program.

This is implemented by unpacking and passing in `sys.argv` to `Game()` when it’s instantiated. The `.__init__()` method uses `*names` to pack the given names into a tuple.

⚠️ Regarding type annotations: even though `names` will be a tuple of strings, **you should only annotate the type of each `name`**. In other words, **you should use `str` and not `Tuple[str]`**:

In [68]:
class Game:
    def __init__(self, *names: str) -> None:
        """Set up the deck and deal cards to 4 players"""
        deck = Deck.create(shuffle=True)
        self.names = (list(names) + "P1 P2 P3 P4".split())[:4]
        self.hands = {
            n: Player(n, h) for n, h in zip(self.names, deck.deal(4))
        }

⚠️ Similarly, if you have a function or method accepting `**kwargs`, then you should only annotate the type of each possible keyword argument.

### Callables

Functions are [first-class objects](https://dbader.org/blog/python-first-class-functions) in Python. This means that you can use functions as arguments to other functions. That also means that you need to be able to add type hints representing functions.

In [69]:
# do_twice.py

from typing import Callable


def do_twice(func: Callable[[str], str], argument: str) -> None:
    print(func(argument))
    print(func(argument))

    
def create_greeting(name: str) -> str:
    return f"Hello {name}"


do_twice(create_greeting, "Jekyll")

Hello Jekyll
Hello Jekyll


#### ⚠️ If you need more flexibility, check out [callback protocols](https://mypy.readthedocs.io/en/latest/protocols.html#callback-protocols).

### Example: Hearts

See `hearts.py`.

Here are a few points to note in the code:
* For type relationships that are hard to express using `Union` or type variables, you can use the `@overload` decorator. See `Deck.__getitem__()` for an example and **[the documentation](https://docs.python.org/3/library/typing.html#typing.overload)** for more information.
* Subclasses correspond to subtypes, so that a `HumanPlayer` can be used wherever a `Player` is expected.
* When a subclass reimplements a method from a superclass, the type annotations must match. See `HumanPlayer.play_card()` for an example.


When starting the game, you control the first player. Enter numbers to choose which cards to play. The following is an example of game play:

```bash
python hearts.py GeirArne Aldren Joanna Brad
```

```
Starting new round:
Brad -> ♣2
  0: ♣5  1: ♣Q  2: ♣K  (Rest: ♢6 ♡10 ♡6 ♠J ♡3 ♡9 ♢10 ♠7 ♠K ♠4)
  GeirArne, choose card: 2
GeirArne => ♣K
Aldren -> ♣10
Joanna -> ♣9
GeirArne wins the trick

  0: ♠4  1: ♣5  2: ♢6  3: ♠7  4: ♢10  5: ♠J  6: ♣Q  7: ♠K  (Rest: ♡10 ♡6 ♡3 ♡9)
  GeirArne, choose card: 0
GeirArne => ♠4
Aldren -> ♠5
Joanna -> ♠3
Brad -> ♠2
Aldren wins the trick

...

Joanna -> ♡J
Brad -> ♡2
  0: ♡6  1: ♡9  (Rest: )
  GeirArne, choose card: 1
GeirArne => ♡9
Aldren -> ♡A
Aldren wins the trick

Aldren -> ♣A
Joanna -> ♡Q
Brad -> ♣J
  0: ♡6  (Rest: )
  GeirArne, choose card: 0
GeirArne => ♡6
Aldren wins the trick

Scores:
Brad             14  14
Aldren           10  10
GeirArne          1   1
Joanna            1   1
```

## Static Type Checking
### Running MyPy

In [70]:
!mypy --help

usage: mypy [-h] [-v] [-V] [more options; see below]
            [-m MODULE] [-p PACKAGE] [-c PROGRAM_TEXT] [files ...]

Mypy is a program that will type check your Python code.

Pass in any files or folders you want to type check. Mypy will
recursively traverse any provided folders to find .py files:

    $ mypy my_program.py my_src_folder

For more information on getting started, see:

- https://mypy.readthedocs.io/en/stable/getting_started.html

For more details on both running mypy and using the flags below, see:

- https://mypy.readthedocs.io/en/stable/running_mypy.html
- https://mypy.readthedocs.io/en/stable/command_line.html

You can also use a config file to configure mypy instead of using
command line flags. For more details, see:

- https://mypy.readthedocs.io/en/stable/config_file.html

Optional arguments:
  -h, --help                Show this help message and exit
  -v, --verbose             More verbose messages
  -V, --version             Show program's version number and

Additionally, the [Mypy command line documentation online](https://mypy.readthedocs.io/en/stable/command_line.html#command-line) has a lot of information.

#### Let’s look at some of the most common options. 

First of all, if you are using third-party packages without type hints, you may want to silence Mypy’s warnings about these. This can be done with the `--ignore-missing-imports` option.

In [71]:
!python cosine.py

[ 1.     0.707  0.    -0.707 -1.    -0.707 -0.     0.707  1.   ]


In [72]:
!mypy cosine.py 

Success: no issues found in 1 source file


**Note:** It appears this error doesn't happen with newer NumPy (presumably it has stub files / typing now).

Tutorial's case:
```bash
$ mypy cosine.py 
cosine.py:3: error: No library stub file for module 'numpy'
cosine.py:3: note: (Stub files are from https://github.com/python/typeshed)
```

These warnings may not immediately make much sense to you, but you’ll learn about [stubs](https://realpython.com/python-type-checking/#adding-stubs) and [typeshed](https://realpython.com/python-type-checking/#typeshed) soon. You can essentially read the warnings as Mypy saying that the Numpy package does not contain type hints.

If you use the `--ignore-missing-import` command line option, Mypy will not try to follow or warn about any missing imports. **This might be a bit heavy-handed though**, as it also ignores actual mistakes, like misspelling the name of a package.

Two less intrusive ways of handling third-party packages are using **type comments** or **configuration files**.

In a simple example as the one above, you can silence the numpy warning by adding a type comment to the line containing the import:

In [73]:
import numpy as np  # type: ignore

If you have several files, it might be easier to keep track of which imports to ignore in a configuration file. Mypy reads a file called `mypy.ini` in the current directory if it is present. This configuration file must contain a section called `[mypy]` and may contain module specific sections of the form `[mypy-module]`.

The following configuration file will ignore that Numpy is missing type hints:

```ini
# mypy.ini

[mypy]

[mypy-numpy]
ignore_missing_imports = True
```

There are many options that can be specified in the configuration file. It is also possible to specify a global configuration file. See the [documentation](https://mypy.readthedocs.io/en/stable/config_file.html) for more information.

### Adding Stubs

The following example uses the Parse package to do simple text parsing. To follow along you should first install Parse:

```bash
$ pip install parse
```

See `parse_name.py`

```bash
$ python parse_name.py
What is your name? I am Geir Arne
Hi Geir Arne, nice to meet you!
```

In [74]:
!mypy parse_name_bug.py

parse_name_bug.py:3: error: Skipping analyzing "parse": found module but no type hints or library stubs
parse_name_bug.py:3: note: See https://mypy.readthedocs.io/en/stable/running_mypy.html#missing-imports
Found 1 error in 1 file (checked 1 source file)


Mypy prints a similar error to the one you saw in the previous section: It doesn’t know about the parse package. You could try to ignore the import:

In [75]:
!mypy parse_name_bug.py --ignore-missing-imports

Success: no issues found in 1 source file


**Unfortunately, ignoring the import means that Mypy has no way of discovering the bug in our program.** 

(A better solution would be to add type hints to the Parse package itself. As Parse is open source you can actually add types to the source code and send a pull request.)

Alternatively, you can add the types in a [stub file](https://mypy.readthedocs.io/en/latest/stubs.html). A stub file is a text file that contains the signatures of methods and functions, but not their implementations. Their main function is to add type hints to code that you for some reason can’t change. To show how this works, we will add some stubs for the Parse package.

First of all, you should put all your stub files inside one common directory, and set the `MYPYPATH` environment variable to point to this directory. 

On Mac and Linux you can set MYPYPATH as follows:

```bash
export MYPYPATH=/home/gahjelle/python/stubs
```

In [76]:
%env MYPYPATH=/mnt/space/Dropbox/Programming/wsl_repos/practice_py/realpython_tutorials/python_type_checking/stubs

env: MYPYPATH=/mnt/space/Dropbox/Programming/wsl_repos/practice_py/realpython_tutorials/python_type_checking/stubs


Next, create a file inside your stubs directory that you call `parse.pyi`. 

It must be named for the package that you are adding type hints for, with a `.pyi` suffix. Leave this file empty for now. 

Then run Mypy again:

In [77]:
!mypy parse_name.py

Success: no issues found in 1 source file


If you have set everything up correctly, you should see this new error message. 

Mypy uses the new `parse.pyi` file to figure out which functions are available in the parse package. Since the stub file is empty, Mypy assumes that `parse.parse()` does not exist, and then gives the error you see above.

The following example does not add types for the whole parse package. Instead it shows the type hints you need to add in order for Mypy to type check your use of `parse.parse()`:

In [78]:
# parse.pyi

from typing import Mapping, Optional, Sequence, Tuple, Union


class Result:
    def __init__(
        self,
        fixed: Sequence[str],
        named: Mapping[str, str],
        spans: Mapping[int, Tuple[int, int]],
    ) -> None: ...
    def __getitem__(self, item: Union[int, str]) -> str: ...
    def __repr__(self) -> str: ...


def parse(
    format: str,
    string: str,
    evaluate_result: bool = ...,
    case_sensitive: bool = ...,
) -> Optional[Result]: ...


The ellipsis `...` are part of the file, and should be written exactly as above. The stub file should only contain type hints for variables, attributes, functions, and methods, so the implementations should be left out and replaced by `...` markers.

Finally Mypy is able to spot the bug we introduced:

In [79]:
!mypy parse_name_bug.py

parse_name_bug.py:18: error: Incompatible return value type (got "Result", expected "str")
Found 1 error in 1 file (checked 1 source file)


😊 This points straight to line 16 and the fact that we return a Result object and not the name string. Change return result back to return `result["name"]`, and run Mypy again to see that it’s happy:

In [81]:
!mypy parse_name.py

Success: no issues found in 1 source file


### Typeshed

[Typeshed](https://github.com/python/typeshed) is a Github repository that contains type hints for the Python standard library, as well as many third-party packages. Typeshed comes included with Mypy so if you are using a package that already has type hints defined in Typeshed, the type checking will just work.

You can also [contribute type hints to Typeshed](https://github.com/python/typeshed/blob/master/CONTRIBUTING.md). Make sure to get the permission of the owner of the package first though, especially because they might be working on adding type hints into the source code itself—which is the [preferred approach](https://github.com/python/typeshed/blob/master/CONTRIBUTING.md#adding-a-new-library).

### Other Static Type Checkers

* [Pyre](https://pyre-check.org/)
* [Pytype](https://github.com/google/pytype)
* See also: https://typing.readthedocs.io/en/latest/#type-checkers

### Using Types at Runtime

As a final note, it’s possible to use type hints also at runtime during execution of your Python program. **Runtime type checking will probably never be natively supported in Python.**

Have a look at [Enforce](https://pypi.org/project/enforce/), [Pydantic](https://pypi.org/project/pydantic/), or [Pytypes](https://pypi.org/project/pytypes/) for some examples.

## Conclusion

Type hinting in Python is a very useful feature that you can happily live without. Type hints don’t make you capable of writing any code you can’t write without using type hints. Instead, using type hints makes it easier for you to reason about code, find subtle bugs, and maintain a clean architecture.

Further reading:
* https://www.python.org/dev/peps/pep-0483/
* https://www.python.org/dev/peps/pep-0484/
* https://mypy.readthedocs.io/